# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [42]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [43]:
output_data_file = "output_data/weather_df.csv"
# output_data_file

types_df = pd.read_csv(output_data_file)
weather_data_df = types_df.drop(columns=["Unnamed: 0"])
weather_data_df

,Cities,Lat,Lng,Country,Date,Cloudiness,Humidity,Max Temp,Wind Speed
0,Walvis Bay,-22.96,14.51,NaN,1587049139,6,52,69.80,9.17
1,Bugene,-1.59,31.14,TZ,1587049139,65,57,70.52,3.65
2,Mataura,-46.19,168.86,NZ,1587049140,97,90,44.01,3.00
3,Troitsko-Pechorsk,62.71,56.20,RU,1587049140,100,78,36.36,8.63
4,Topaowan,30.67,112.69,CN,1587049140,95,73,69.13,12.86
...,...,...,...,...,...,...,...,...,...
571,Maba,24.68,113.60,CN,1587049305,0,71,68.02,3.04
572,Thinadhoo,0.53,72.93,MV,1587049305,96,68,85.33,6.35
573,Constantine,36.37,6.61,DZ,1587049305,75,34,78.80,3.36
574,Qandala,11.47,49.87,SO,1587049305,4,63,85.96,5.95


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [44]:
gmaps.configure(api_key=g_key)


In [45]:
# Use Lat and Lng as locations
location=weather_data_df[["Lat","Lng"]]
# Use humidity as the weigtht
humidity=weather_data_df["Humidity"].astype(float)
# Add Heatmap layer to map
fig =gmaps.figure()
heatmap_layer=gmaps.heatmap_layer(location, weights=humidity, dissipating =False, max_intensity=15, point_radius=2)
fig.add_layer(heatmap_layer)
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [46]:
# Narrow down the DataFrame to find your ideal weather condition.
Narrow_df=pd.DataFrame(weather_data_df, columns=["Cities", "Max Temp", "Cloudiness", "Wind Speed","Lat","Lng","Country"])
max_temp=(Narrow_df["Max Temp"] <=80) & (Narrow_df["Max Temp"] >=70)
cloudiness = Narrow_df["Cloudiness"] == 0
Wind_Speed=Narrow_df["Wind Speed"] < 10
# Filter the ideal cities
narrowed_city_df= Narrow_df[max_temp & cloudiness & Wind_Speed]
# Drop any rows that don't contain all three conditions. 
narrowed_city_df

,Cities,Max Temp,Cloudiness,Wind Speed,Lat,Lng,Country
54,Bereket,75.63,0,4.23,39.24,55.52,TM
109,Rio Verde de Mato Grosso,75.11,0,6.20,-18.92,-54.84,BR
187,Benjamín Aceval,73.40,0,5.82,-24.97,-57.57,PY
192,Castro,71.37,0,2.33,-24.79,-50.01,BR
355,Resistencia,75.99,0,5.82,-27.46,-58.98,AR
384,Caarapó,70.99,0,7.52,-22.63,-54.82,BR
428,Gold Coast,70.00,0,8.05,-28.00,153.43,AU
475,Tupi Paulista,70.90,0,4.92,-21.38,-51.57,BR
478,Luján,70.00,0,4.70,-34.57,-59.10,AR
504,Três Lagoas,71.46,0,5.50,-20.75,-51.68,BR


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [47]:
# Store into variable named hotel_df.
# Add a "Hotel Name" column to the DataFrame.
hotel_df = pd.DataFrame(narrowed_city_df, columns=["Cities", "Country", "Lat", "Lng"])
hotel_df["Hotel Name"]=""
hotel_df.head()

,Cities,Country,Lat,Lng,Hotel Name
54,Bereket,TM,39.24,55.52,
109,Rio Verde de Mato Grosso,BR,-18.92,-54.84,
187,Benjamín Aceval,PY,-24.97,-57.57,
192,Castro,BR,-24.79,-50.01,
355,Resistencia,AR,-27.46,-58.98,


In [49]:
hotel_df

,Cities,Country,Lat,Lng,Hotel Name
54,Bereket,TM,39.24,55.52,
109,Rio Verde de Mato Grosso,BR,-18.92,-54.84,
187,Benjamín Aceval,PY,-24.97,-57.57,
192,Castro,BR,-24.79,-50.01,
355,Resistencia,AR,-27.46,-58.98,
384,Caarapó,BR,-22.63,-54.82,
428,Gold Coast,AU,-28.00,153.43,
475,Tupi Paulista,BR,-21.38,-51.57,
478,Luján,AR,-34.57,-59.10,
504,Três Lagoas,BR,-20.75,-51.68,


In [54]:
# creates a list to store hotel names
hotel_name = []
# parameters dictionary
params = {
    "radius":5000,
    "types":"lodging",
    "keyword":"hotel",
    "key": g_key
}
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address['results'][0]['name']
        #hotel_name.append(name_address['results'][0]['name'])
    except (KeyError, IndexError):
         print('missing infor')
            #hotel_name.append(np.nan)   

missing infor
missing infor


In [55]:
hotel_df

,Cities,Country,Lat,Lng,Hotel Name
54,Bereket,TM,39.24,55.52,
109,Rio Verde de Mato Grosso,BR,-18.92,-54.84,Quedas Palace Hotel
187,Benjamín Aceval,PY,-24.97,-57.57,Escuela Agrícola y Hotel Cerrito
192,Castro,BR,-24.79,-50.01,Hotel Buganville Express
355,Resistencia,AR,-27.46,-58.98,Niyat Urban Hotel
384,Caarapó,BR,-22.63,-54.82,Hotel Pousada Quinta do Sul
428,Gold Coast,AU,-28.00,153.43,Hilton Surfers Paradise Hotel & Residences
475,Tupi Paulista,BR,-21.38,-51.57,Hotel Tupi
478,Luján,AR,-34.57,-59.10,Real Hotel Luján
504,Três Lagoas,BR,-20.75,-51.68,Hotel OT


In [61]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{Cities}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [62]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
# Display Map
fig

Figure(layout=FigureLayout(height='420px'))